In [3]:
from langchain_openai import ChatOpenAI
import os 
from crewai_tools import PDFSearchTool,tool
from crewai import Crew,Task,Agent,Process
from langchain_community.tools.tavily_search import TavilySearchResults


In [4]:
os.environ['GROQ_API_KEY'] = "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"


In [5]:
llm = ChatOpenAI(
    openai_api_base = "https://api.groq.com/openai/v1",
    openai_api_key = os.environ['GROQ_API_KEY'],
    model_name = "llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000
)

In [9]:
rag_tool = PDFSearchTool(pdf = 'attention.pdf',
                         config = dict(
                             llm = dict(
                                 provider = "groq",
                                 config = dict(
                                    model = "llama3-8b-8192"
                                 ),
                             
                            ),
                         embedder = dict(provider = "huggingface",
                                         config = dict(
                                             model = "BAAI/bge-small-en-v1.5"
                                         )
                                         )
                         )
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\dhanu\.conda\envs\lm_python\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dhanu\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\dhanu\.conda\envs\lm_python\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


In [10]:
rag_tool.run("Explain the Attention mechanism")

Using Tool: Search a PDF's content


'Relevant Content:\nthe foundation of the Extended Neural GPU [16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difficult to learn dependencies between distant positions [ 12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been use

In [11]:
os.environ['TAVILY_API_KEY'] = "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"

In [12]:
web_Search_tool = TavilySearchResults(k=3)

In [13]:
web_Search_tool.run("Explain the Attention mechanism")

[{'url': 'https://machinelearningmastery.com/the-attention-mechanism-from-scratch/',
  'content': 'The General Attention Mechanism\nThe general attention mechanism makes use of three main components, namely the queries, $\\mathbf{Q}$, the keys, $\\mathbf{K}$, and the values, $\\mathbf{V}$.\nIf you had to compare these three components to the attention mechanism as proposed by Bahdanau et al., then the query would be analogous to the previous decoder output, $\\mathbf{s}_{t-1}$, while the values would be analogous to the encoded inputs, $\\mathbf{h}_i$. In the Bahdanau attention mechanism, the keys and values are the same vector.\n But if we have thought for example the first row of the score matrix as:\n[ word1/word1 word1/word2 word1/word3 word1/word4 ]\nthen the elements of the attention vector (a row of the attention matrix) might be interpreted as somewhat a coded version of the row of the score matrix since in the product between two matrix we do rows*columns product.\n In the lat

In [14]:
@tool 
def router_tool(question):
    """ Router Function"""
    if 'Attention' in question:
        return 'vectorstore'
    else:
        return 'web_search'

In [16]:
Router_agent = Agent(
    role = 'Router',
    goal = 'Route user questions to a vector store or web search',
    backstory = (
        "You are an expert at routing a user questions to a vectorstore or a websearch."
        "Use the vectorstores for questions on concept relevant to RAG."
        "You don not need to be stringest with the keywords in the question related to the topics"

    ),
    verbose = True,
    allow_delegation = True,
    llm = llm

)

In [17]:
Retriever_Agent = Agent(role = "Retriever",
                        goal = "USe the information retrieved from the vectorstores to answer the questions",
                        backstory = (
                            "You are an expert at using information from vector stores to answer questions."
                            "You use the vector stores to find relevant documents."
                            "You don't need to be stringent with the keywords in the question related to the topics"
                        ),
                        verbose = True,
                        allow_delegation = True,
                        llm = llm)

In [18]:
Grader_Agent = Agent(
    role = "Answer Grader",
    goal = "Grade the answers provided by the Retriever Agent",
    backstory = (
        "You are an expert at grading answers provided by the Retriever Agent."
        "You assess the accuracy of the answers."
        "You don't need to be stringent with the keywords in the question related to the topics"
),
verbose = True,
allow_delegation = True,
llm = llm
)

In [19]:
hallucination_grader = Agent(role = "Hallucination Grader",
                             goal = "Filter out hallucination",
                             backstory = (
                                 "You are a hallucination grader assessing whether an answer is hallucinated or not and Make sure that you metriculously review the answer and check if the results are correct"
                             ),
                             verbose = True,
                             allow_delegation = True,
                             llm = llm)

In [20]:
answer_grader = Agent(role = "Answer Grader",
                      goal = "Filter out hallucinations from the answer",
                      backstory = (
                          "You are an answer grader assessing whether an answer is hallucinated or not and Make sure that you metriculously review the answer and check if the results are correct"
                          "Make sure you metriculously review answerr and check if it makes sense to the questions"
                      ),
                      verbose = True,
                      allow_delegation = True,
                      llm = llm)

In [22]:

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [23]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [24]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [25]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [26]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [27]:

router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [28]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [29]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [30]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

In [31]:
answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [32]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)


In [33]:

inputs ={"question":"Explain the attention mechanism"}

In [34]:


result = rag_crew.kickoff(inputs=inputs)

 [2024-07-18 17:29:18][DEBUG]: == Working Agent: Router
 [2024-07-18 17:29:18][INFO]: == Starting Task: Analyse the keywords in the question Explain the attention mechanismBased on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to analyze the keywords in the question to determine whether it is eligible for a vectorstore search or a web search.

Action: router_tool

Action Input: {"question": "Explain the attention mechanism"}
 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool router_tool accepts these inputs: router_tool() -  Router Function

Thought: I need to analyze the keywords in the question 